In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split


In [ ]:

def logistic_func(x):

    L = 1 / (1 + np.exp(-x))
    return L

In [ ]:
def train(X_train, y_train, tol=10 ** -4):

    LearningRate = 0.05


    n_features = X_train.shape[1]
    n_sample = X_train.shape[0]

    weights = 2 * np.random.random_sample((n_features + 1, 1)) - 1  # random[-1,1)

    print(
        "It is normal to see: 'ex1.py:13: RuntimeWarning: overflow encountered in exp'"
    )
    print("Please wait for a while for traning, sometimes fast, sometimes slow")
    round = 1
    while True:
        print(f"The {round}th round")
        # initialize delta
        delta = [0 for i in range(0, n_features + 1)]

        for t in range(1, n_sample + 1):
            g = 0

            g = g + weights[0]  # add back w_0

            for i in range(1, n_features + 1):

                g = g + weights[i] * X_train[t - 1, i - 1]

            L = logistic_func(g)

            # add delta_0
            old_zero = delta[0]
            delta[0] = delta[0] + (y_train[t - 1] - L)  # add back delta w_0

            for i in range(1, n_features + 1):
                old_delta = delta[i]
                delta[i] = delta[i] + (y_train[t - 1] - L) * X_train[t - 1, i - 1]

        # for storing old weight of each round
        old_wieght = np.copy(weights)

        for i in range(0, n_features + 1):
            weights[i] = weights[i] + LearningRate * delta[i]

        # calculation convergence
        converge = 0

        for t in range(0, n_features + 1):
            converge = converge + abs(weights[t] - old_wieght[t])


        print(f"converge value is {converge}")

        if converge < tol:
            break
        else:
            round += 1


    return weights

In [ ]:
def train_matrix(X_train, y_train, tol=10 ** -4):

    print(
        "It is normal to see: 'ex1.py:13: RuntimeWarning: overflow encountered in exp'"
    )
    print("Please wait for a while for traning, sometimes fast, sometimes slow")
    LearningRate = 0.05
    n_features = X_train.shape[1]

    weights = 2 * np.random.random_sample((n_features + 1, 1)) - 1  # random[-1,1)

    x_train = np.empty((X_train.shape[0], X_train.shape[1] + 1))
    # x_train = (750x3)
    row = 0
    for x in X_train:

        x = np.insert(x, 0, 1, axis=0)
        x_train[row] = x
        row += 1

    y_train = np.transpose(np.array([y_train]))

    round = 1
    while True:
        print(f"The {round}th round")
        old_weights = np.copy(weights)
        mul = np.matmul(x_train, weights)
        take_logistics = logistic_func(mul)
        subtract_log = y_train - take_logistics
        delta = np.matmul(np.transpose(x_train), subtract_log)
        weights = weights + delta * LearningRate
        converge = np.sum(np.absolute(weights - old_weights))

        print(f"converge value is {converge}")

        if converge < tol:
            break
        else:
            round += 1

    return weights

In [ ]:
def predict(X_test, weights):

    predictions = []
    n_samples = X_test.shape[0]

    for i in range(0, n_samples):
        prbability = logistic_func(
            np.matmul(np.transpose(weights[1:]), X_test[i]) + weights[0]
        )
        #classification
        if prbability > 0.5:
            predictions.append(1)
        else:
            predictions.append(0)
    predictions = np.array(predictions)
    return predictions

In [ ]:
def plot_prediction(X_test, X_test_prediction):
    X_test1 = X_test[X_test_prediction == 0, :]
    X_test2 = X_test[X_test_prediction == 1, :]
    plt.scatter(X_test1[:, 0], X_test1[:, 1], color="red")
    plt.scatter(X_test2[:, 0], X_test2[:, 1], color="blue")
    plt.show()

In [ ]:
# Data Generation
n_samples = 1000

centers = [(-1, -1), (5, 10)]
X, y = make_blobs(
    n_samples=n_samples,
    n_features=2,
    cluster_std=1.8,
    centers=centers,
    shuffle=False,
    random_state=42,
)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Experiments
w = train(X_train, y_train)
# w = train_matrix(X_train, y_train)
X_test_prediction = predict(X_test, w)
plot_prediction(X_test, X_test_prediction)
plot_prediction(X_test, y_test)

wrong = np.count_nonzero(y_test - X_test_prediction)
print("Number of wrong predictions is: " + str(wrong))

# np.array([-111.83560223,19.15577369, 16.4830988])
